# 📘 Sesión 6 — Joins y Uniones de DataFrames en PySpark

En la práctica real, los datos están distribuidos en múltiples tablas. Para analizarlos en conjunto, necesitamos combinarlos. Esto se logra con los **joins**, también conocidos como uniones de DataFrames.

---

## 🔧 Tipos de joins en PySpark

| Tipo de Join     | Descripción                                              |
|------------------|----------------------------------------------------------|
| `inner`          | Devuelve solo las filas con claves coincidentes         |
| `left`           | Devuelve todas las filas de la izquierda y las coincidentes de la derecha |
| `right`          | Devuelve todas las filas de la derecha y las coincidentes de la izquierda |
| `outer` (full)   | Devuelve todas las filas, coincidan o no                |
| `semi` / `anti`  | Filtros especializados según existencia de coincidencia |

---

## ✨ ¿Cuándo se usan?

- Para enriquecer una tabla con información adicional (cliente + transacción)
- Para detectar registros sin coincidencias
- Para construir un dataset unificado para análisis o modelos

---

In [0]:
from pyspark.sql import functions as F

# Crear DataFrame de clientes
df_clientes = spark.createDataFrame([
    (1, "Ana"),
    (2, "Beto"),
    (3, "Carla"),
    (4, "David")
], ["customerID", "nombre"])

# Crear DataFrame de transacciones
df_transacciones = spark.createDataFrame([
    (100, 1, 150),
    (101, 2, 200),
    (102, 5, 300),
    (103, 6, 120)
], ["transactionID", "customerID", "monto"])

# INNER JOIN: solo clientes con transacciones
print("📌 INNER JOIN")
df_inner = df_clientes.join(df_transacciones, on="customerID", how="inner")
df_inner.show()
# LEFT JOIN: todos los clientes, con o sin transacciones
print("📌 LEFT JOIN")
df_left = df_clientes.alias("x").join(
            df_transacciones.alias("y"), 
            on="customerID", 
            how="left"
            )
df_left.show()        

# RIGHT JOIN: todas las transacciones, incluso si no hay cliente
print("📌 RIGHT JOIN")
df_right = df_clientes.alias("x").join(
                df_transacciones.alias("y"), 
                F.col("x.customerID")==F.col("y.customerID"), 
                how="right").select(
                    F.col("x.*"),
                    F.col("y.transactionID"),
                    F.col("y.monto")
                )
df_right.show()

# OUTER JOIN: todos los registros de ambos DataFrames
print("📌 OUTER JOIN")
sql_query="""
select 
x.*, 
y.transactionID, y.monto
from {clientes} x
full outer join {transacciones} y ON x.customerID=y.customerID
"""
df_outer = spark.sql(sql_query, clientes=df_clientes, transacciones=df_transacciones)
df_outer.show()

 Script en PySpark — Ejemplo de UNION y UNION ALL con datos inventados

In [0]:
# Crear el primer DataFrame (clientes registrados en enero)
df_enero = spark.createDataFrame([
    (1, "Ana", "Mexico"),
    (2, "Luis", "Argentina"),
    (3, "Camila", "Chile")
], ["customerID", "nombre", "pais"])

# Crear el segundo DataFrame (clientes registrados en febrero)
df_febrero = spark.createDataFrame([
    (3, "Camila", "Chile"),         # repetido
    (4, "Pedro", "Colombia"),
    (5, "Lucía", "Perú")
], ["customerID", "nombre", "pais"])

# UNION: elimina duplicados automáticamente
df_union = df_enero.union(df_febrero).distinct()

# UNION ALL: conserva todos los registros, incluyendo duplicados
df_union_all = df_enero.union(df_febrero)

# Mostrar resultados
print("📌 UNION (sin duplicados):")
df_union.show()

print("📌 UNION ALL (con duplicados):")
df_union_all.show()

✍️ Ejercicio — Inner Join entre franquicias y transacciones

📘 Objetivo

Combinar la información de las franquicias con las transacciones de venta, para analizar cuántas operaciones se realizaron por país. Vamos a usar un inner join entre las tablas:
- samples.bakehouse.sales_franchises
- samples.bakehouse.sales_transactions

🗂️ Esquema de columnas relevantes

sales_franchises
```sh
	•	franchiseID
	•	country
	•	city
	•	state
  ```

sales_transactions

```sh
	•	transactionID
	•	franchiseID
	•	dateTime
	•	totalPrice
	•	product
``` 

🎯 Enunciado del ejercicio:

- 1 Realizá un inner join entre sales_franchises y sales_transactions usando la columna franchiseID como clave.
- 2 Calculá cuántas transacciones se realizaron por país (country).
- 3 Mostrá los resultados en un DataFrame con las columnas:
  - country
  - rowsn (cantidad de transacciones)
- 4 Ordená los resultados de forma descendente según el número de transacciones.